In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import warnings
warnings.filterwarnings('ignore')


In [2]:
heart_df=pd.read_csv("resources/cleaner_bmi.csv")
heart_df.head()

,id,age (years),gender,BMI,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
0,0,50,2,22.015308,110,80,1,1,0,0,1,0
1,1,55,1,34.850994,140,90,3,1,0,0,1,1
2,2,51,1,23.489511,130,70,3,1,0,0,0,1
3,3,48,2,28.742724,150,100,1,1,0,0,1,1
4,4,47,1,22.923381,100,60,1,1,0,0,0,0


In [3]:
y = heart_df["cardio"]
X = heart_df.drop(["id","cardio"], axis=1)

In [4]:
X.describe()

,age (years),gender,BMI,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
count,65839.000000,65839.000000,65839.000000,65839.000000,65839.000000,65839.000000,65839.000000,65839.000000,65839.000000,65839.000000
mean,52.820562,1.350385,27.021492,126.259011,81.153845,1.356141,1.219171,0.087592,0.052507,0.804523
std,6.768891,0.477094,4.508784,16.026882,9.121909,0.672405,0.564869,0.282703,0.223049,0.396570
min,29.000000,1.000000,14.495481,90.000000,60.000000,1.000000,1.000000,0.000000,0.000000,0.000000
25%,48.000000,1.000000,23.822696,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000
50%,53.000000,1.000000,26.193395,120.000000,80.000000,1.000000,1.000000,0.000000,0.000000,1.000000
75%,58.000000,2.000000,29.728700,140.000000,90.000000,1.000000,1.000000,0.000000,0.000000,1.000000
max,64.000000,2.000000,48.105155,197.000000,126.000000,3.000000,3.000000,1.000000,1.000000,1.000000


In [5]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=78)
X_train.shape

(49379, 10)

In [6]:
heart_df["cardio"].value_counts()

0    33661
1    32178
Name: cardio, dtype: int64

In [7]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [8]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh'])
    
    # Allow kerastuner to decide number of neurons in first layer
    number_of_nerons = hp.Int('first_units', min_value=1, max_value=30, step=5)
    nn_model.add(tf.keras.layers.Dense(units=number_of_nerons, activation=activation, input_dim=10))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 5)):
        number_of_nerons = hp.Int('units_' + str(i), min_value=1, max_value=30, step=5)
        nn_model.add(tf.keras.layers.Dense(units=number_of_nerons, activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [9]:
# Import the kerastuner library
# import kerastuner as kt
import keras_tuner as kt

tuner = kt.Hyperband(
    create_model, # function name
    objective="val_accuracy",
    max_epochs=50,
    hyperband_iterations=2)

In [10]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

Trial 180 Complete [00h 02m 15s]
val_accuracy: 0.7408869862556458

Best val_accuracy So Far: 0.7430133819580078
Total elapsed time: 06h 23m 12s
INFO:tensorflow:Oracle triggered exit


In [11]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 21,
 'num_layers': 3,
 'units_0': 11,
 'units_1': 16,
 'units_2': 16,
 'units_3': 1,
 'units_4': 11,
 'tuner/epochs': 50,
 'tuner/initial_epoch': 17,
 'tuner/bracket': 1,
 'tuner/round': 1,
 'tuner/trial_id': '0077'}

In [12]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

515/515 - 1s - loss: 0.5370 - accuracy: 0.7430 - 575ms/epoch - 1ms/step
Loss: 0.5370136499404907, Accuracy: 0.7430133819580078
